# Think Green Investing 

## Data Preparation

In [1]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import panel as pn
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
import os
from pathlib import Path
import plotly.graph_objects as go
import plotly.offline as pyo
from finta import TA
import talib as ta 

%matplotlib inline

In [2]:
#intializing alpaca trade api
load_dotenv()
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')
alpaca=tradeapi.REST(alpaca_api_key,alpaca_secret_key, api_version='v2')

In [3]:
#function that pulls tickers daily closing prices in the given time period using alpaca trade api, 
#intialization of the api is required before using.
#It returns a dataframe with the tickers, and closing prices as two level column structured datafram with index defined as date  

def get_tickers_data(tickers, start_date, end_date, timeframe):
    start_date = pd.Timestamp(start_date, tz = "America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz = "America/New_York").isoformat()
    tickers_data_df = alpaca.get_bars(tickers, timeframe,start_date , end_date, adjustment = 'raw').df
    tickers_data_df.index = tickers_data_df.index.date
    tickers_data_df = tickers_data_df[['symbol', 'open', 'high', 'low','close']]
    tickers_data_list=[]
    for ticker in tickers:
        tickers_data_list.append(tickers_data_df[tickers_data_df['symbol']==ticker].drop('symbol', axis=1))
        
    clean_tickers_data = pd.concat(tickers_data_list,axis=1, keys=tickers)
    #clean_tickers_data.columns=clean_tickers_data.columns.droplevel(-1)
    return clean_tickers_data

In [4]:
#defining the stocks in the solar energy portfolio form top performers in 2021 using mootley fool and investopedia. The companies are:
#1.First Solar (FSLR)
#2.Brookfield Renewable (BEPC) --> removed 
#3.Solar Edge Technologies (SEDG)
#4.Daqo New Energy Corp. (DQ)
#5.Canadian Solar INC (CSIQ)


# Set the ticker information
portfolio = ['FSLR' , 'SEDG', 'DQ', 'CSIQ', 'AQN']
indices = ['SPY']
timeframe='1Day'
tickers = portfolio + indices

Portfolio### Five Year Period Jan 2018 to April 2022

In [5]:
start_date = '2019-01-01'
end_date = '2022-06-15'

tickers_data = get_tickers_data(tickers, start_date, end_date, timeframe) 



In [6]:
tickers_data.dropna(inplace = True)
tickers_data

FSLR                            SEDG                              \
             open     high     low  close    open      high       low   close   
2019-01-02  41.63  43.1000  41.410  42.56   34.41   36.4450   34.3500   36.15   
2019-01-03  43.97  44.6316  42.700  43.44   35.68   37.0678   35.5000   35.50   
2019-01-04  43.98  46.6000  43.980  46.32   35.91   37.3800   35.2200   37.25   
2019-01-07  46.46  47.7400  45.660  45.79   37.20   37.2000   34.9200   35.01   
2019-01-08  46.80  46.9999  45.350  46.23   35.36   35.7366   34.0500   35.00   
...           ...      ...     ...    ...     ...       ...       ...     ...   
2022-06-09  70.93  71.8600  69.770  69.83  296.14  302.0000  291.3400  291.67   
2022-06-10  68.23  68.9500  67.070  68.00  286.56  292.0000  278.5800  284.36   
2022-06-13  65.14  66.0600  62.770  63.13  269.47  271.5400  255.8250  264.34   
2022-06-14  63.50  64.1500  61.075  61.95  265.33  269.5600  260.4201  264.63   
2022-06-15  62.50  64.4900  61.510  64.03  267.36  277.3800  260.8450  274.13   

               DQ           ...   CSIQ           AQN                          \
             open     high  ...    low  close   open    high      low  close   
2019-01-02  23.01  24.4100  ...  14.00  14.44  10.01  10.030   9.9000   9.96   
2019-01-03  23.82  24.1014  ...  14.80  15.14   9.99  10.090   9.9000  10.04   
2019-01-04  23.82  25.2100  ...  15.20  16.37  10.10  10.210  10.0000  10.10   
2019-01-07  25.05  26.9300  ...  15.96  16.80  10.19  10.270  10.1300  10.26   
2019-01-08  26.93  27.3400  ...  16.82  17.13  10.30  10.390  10.2700  10.38   
...           ...      ...  ...    ...    ...    ...     ...      ...    ...   
2022-06-09  59.90  63.2900  ...  33.96  34.02  14.57  14.635  14.2800  14.30   
2022-06-10  60.00  65.3700  ...  32.32  32.34  14.15  14.245  14.0525  14.18   
2022-06-13  58.81  59.9700  ...  29.50  30.16  13.99  14.010  13.7250  13.82   
2022-06-14  59.71  59.8400  ...  29.14  29.37  13.78  13.790  13.1500  13.19   
2022-06-15  57.33  58.4200  ...  28.69  29.78  13.27  13.500  13.1000  13.35   

               SPY                            
              open      high     low   close  
2019-01-02  246.06  251.2100  245.95  250.18  
2019-01-03  248.30  248.5700  243.67  244.21  
2019-01-04  247.59  253.1100  247.17  252.39  
2019-01-07  252.76  255.9500  251.69  254.38  
2019-01-08  256.86  257.3100  254.00  256.77  
...            ...       ...     ...     ...  
2022-06-09  409.36  411.7400  401.44  401.44  
2022-06-10  394.88  395.7777  389.75  389.80  
2022-06-13  380.00  381.8100  373.30  375.00  
2022-06-14  376.84  377.9400  370.59  373.87  
2022-06-15  377.33  383.9000  372.12  379.20  

[871 rows x 24 columns]

In [22]:
# Slice to just the `close` column
def algo_trading(close_df): 
   
    # Determine the Bollinger Bands for the Dataset
    bbands_df = TA.BBANDS(close_df)

    # Concatenate the Bollinger Bands to the DataFrame
    bb_df = pd.concat([close_df, bbands_df], axis=1)

    # Set the Signal column
    bb_df["Signal"] = 0.0
#RSI calculation
    bb_df['RSI'] = ta.RSI(bb_df['close'], 14)
# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm 
    for index, row in bb_df.iterrows():
        if row["close"] < row["BB_LOWER"] and row["RSI"] < row["BB_LOWER"]:
            bb_df.loc[index, "Signal"] = 1.0
        if row["close"] > row["BB_UPPER"] and row["RSI"] > row["BB_UPPER"]:
            bb_df.loc[index,"Signal"] = -1.0
    return bb_df


In [23]:
# Visualize entry position relative to close price
def visualize_algo_trading(bb_df): 

    entry = bb_df[bb_df["Signal"] == 1.0]["close"].hvplot.scatter(
        color="green",
        marker="^",
        size=200,
        legend=False,
        ylabel="Price in $",
        width=1000,
        height=400
    )

# Visualize exit position relative to close price
    exit = bb_df[bb_df["Signal"] == -1.0]["close"].hvplot.scatter(
        color="red",
        marker="v",
        size=200,
        legend=False,
        ylabel="Price in $",
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = bb_df[["close"]].hvplot(
        line_color="gray",
        ylabel="Price in $",
        width=1000,
        height=400
    )

    bb_upper = bb_df[["BB_UPPER"]].hvplot(
        line_color="purple",
        ylabel="Price in $",
        width=1000,
        height=400
    )


    bb_middle = bb_df[["BB_MIDDLE"]].hvplot(
        line_color="orange",
        ylabel="Price in $",
        width=1000,
        height=400
    )

    bb_lower = bb_df[["BB_LOWER"]].hvplot(
        line_color="blue",
        ylabel="Price in $",
        width=1000,
        height=400
    )


# Overlay plots
    bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
    return  bbands_plot

In [24]:
def cal_return(initial_capital,bb_df):
   
    # Set the share size
    share_size = 100

    # Buy a 100 share position when the dual moving average crossover Signal equals 1
    # Otherwise, `Position` should be zero (sell)
    bb_df['Position'] = share_size * bb_df['Signal']

    # Determine the points in time where a 100 share position is bought or sold
    bb_df['Entry/Exit Position'] = bb_df['Position'].diff()

    # Multiply the close price by the number of shares held, or the Position
    bb_df['Portfolio Holdings'] = bb_df['close'] * bb_df['Position']

    # Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
    bb_df['Portfolio Cash'] = initial_capital - (bb_df['close'] * bb_df['Entry/Exit Position']).cumsum() 

    # Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
    bb_df['Portfolio Total'] = bb_df['Portfolio Cash'] + bb_df['Portfolio Holdings']

    # Calculate the portfolio daily returns
    bb_df['Portfolio Daily Returns'] = bb_df['Portfolio Total'].pct_change()

    # Calculate the portfolio cumulative returns
    bb_df['Portfolio Cumulative Returns'] = (1 + bb_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
    return bb_df

In [25]:
capital = 1e5
FSLR_bb_df = algo_trading(tickers_data['FSLR'])
plot_tr=visualize_algo_trading(FSLR_bb_df)   
FSLR_bb_df = cal_return(capital,FSLR_bb_df)


In [26]:
print(FSLR_bb_df['Portfolio Total'][-1])
plot_tr

104078.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [27]:
capital = 1e5
SEDG_bb_df = algo_trading(tickers_data['SEDG'])
plot_tr=visualize_algo_trading(SEDG_bb_df)   
SEDG_bb_df = cal_return(capital,SEDG_bb_df)


In [28]:
print(SEDG_bb_df['Portfolio Total'][-1])
plot_tr

99902.5


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [29]:
capital = 1e5
DQ_bb_df = algo_trading(tickers_data['DQ'])
plot_tr=visualize_algo_trading(DQ_bb_df)   
DQ_bb_df = cal_return(capital,DQ_bb_df)


In [30]:
print(DQ_bb_df['Portfolio Total'][-1])
plot_tr

100616.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [31]:
capital = 1e5
CSIQ_bb_df = algo_trading(tickers_data['CSIQ'])
plot_tr=visualize_algo_trading(CSIQ_bb_df)   
CSIQ_bb_df = cal_return(capital,CSIQ_bb_df)


In [33]:
print(CSIQ_bb_df['Portfolio Total'][-1])
plot_tr

100361.73


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [34]:
capital = 1e5
AQN_bb_df = algo_trading(tickers_data['AQN'])
plot_tr=visualize_algo_trading(AQN_bb_df)   
AQN_bb_df = cal_return(capital,AQN_bb_df)


In [35]:
print(AQN_bb_df['Portfolio Total'][-1])
plot_tr

99823.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [36]:
capital = 1e5
SPY_bb_df = algo_trading(tickers_data['SPY'])
plot_tr=visualize_algo_trading(SPY_bb_df)   
SPY_bb_df = cal_return(capital,SPY_bb_df)


In [37]:
print(SPY_bb_df['Portfolio Total'][-1])
plot_tr

106960.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)